In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import random as rd
import os
from missingpy import MissForest
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from scipy import stats
import statsmodels as sm
from statsmodels.stats.weightstats import ztest
from statsmodels.imputation import mice
import plotly

In [3]:
%matplotlib inline

# @grellert: removendo warnings chatos
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")


In [4]:
df = pd.read_csv(os.path.join('D:/', 'bolsa PIBIC', 'Covid19_Data', 'csv', 'worldometer_coronavirus_summary_data.csv'))
df

,country,continent,total_confirmed,total_deaths,total_recovered,active_cases,serious_or_critical,total_cases_per_1m_population,total_deaths_per_1m_population,total_tests,total_tests_per_1m_population,population
0,Afghanistan,Asia,147473,6736.0,98073.0,42664.0,1124.0,3699,169.0,707244.0,17739.0,39869084
1,Albania,Europe,133081,2457.0,130222.0,402.0,3.0,46299,855.0,916777.0,318945.0,2874408
2,Algeria,Africa,171392,4254.0,115276.0,51862.0,47.0,3834,95.0,230861.0,5164.0,44701619
3,Andorra,Europe,14678,128.0,14210.0,340.0,NaN,189643,1654.0,193595.0,2501292.0,77398
4,Angola,Africa,42646,1008.0,36708.0,4930.0,8.0,1256,30.0,681443.0,20065.0,33961015
...,...,...,...,...,...,...,...,...,...,...,...,...
215,Wallis And Futuna Islands,Australia/Oceania,445,7.0,438.0,0.0,NaN,40359,635.0,20508.0,1859967.0,11026
216,Western Sahara,Africa,10,1.0,8.0,1.0,NaN,16,2.0,NaN,NaN,613090
217,Yemen,Asia,7061,1375.0,4185.0,1501.0,23.0,231,45.0,152937.0,5009.0,30530919
218,Zambia,Africa,195816,3389.0,187236.0,5191.0,422.0,10341,179.0,2089592.0,110352.0,18935680


In [5]:
df = df.dropna()
print(df)

                 country      continent  total_confirmed  total_deaths  \
0            Afghanistan           Asia           147473        6736.0   
1                Albania         Europe           133081        2457.0   
2                Algeria         Africa           171392        4254.0   
4                 Angola         Africa            42646        1008.0   
6    Antigua And Barbuda  North America             1303          43.0   
..                   ...            ...              ...           ...   
211           Uzbekistan           Asia           129327         874.0   
213            Venezuela  South America           304726        3576.0   
217                Yemen           Asia             7061        1375.0   
218               Zambia         Africa           195816        3389.0   
219             Zimbabwe         Africa           107490        3490.0   

     total_recovered  active_cases  serious_or_critical  \
0            98073.0       42664.0               112

In [6]:
np.where(pd.isnull(df))

(array([], dtype=int64), array([], dtype=int64))

In [7]:
df.describe()

,total_confirmed,total_deaths,total_recovered,active_cases,serious_or_critical,total_cases_per_1m_population,total_deaths_per_1m_population,total_tests,total_tests_per_1m_population,population
count,1.440000e+02,144.000000,1.440000e+02,1.440000e+02,144.000000,144.000000,144.000000,1.440000e+02,1.440000e+02,1.440000e+02
mean,1.307081e+06,26832.638889,1.184411e+06,9.583686e+04,597.895833,48260.590278,806.881944,1.955644e+07,9.031496e+05,4.726722e+07
std,4.386056e+06,83171.849105,3.955494e+06,4.729728e+05,1632.836961,40317.198544,772.783946,6.523976e+07,1.677604e+06,1.704177e+08
min,5.890000e+02,3.000000,1.040000e+02,-1.489000e+03,1.000000,65.000000,3.000000,1.714700e+04,5.009000e+03,3.044800e+04
25%,3.404525e+04,507.000000,3.120325e+04,1.206000e+03,9.000000,10647.000000,123.000000,3.612762e+05,1.159155e+05,2.063909e+06
50%,1.970805e+05,2901.000000,1.695775e+05,9.814500e+03,36.500000,42805.000000,607.000000,2.082408e+06,4.277675e+05,9.227520e+06
75%,6.737082e+05,14695.000000,6.552955e+05,3.883925e+04,307.000000,80596.750000,1226.250000,1.142994e+07,1.015640e+06,3.396843e+07
max,3.573397e+07,629290.000000,3.081287e+07,5.438593e+06,9375.000000,162265.000000,3117.000000,5.295773e+08,1.311142e+07,1.439324e+09


In [8]:
TH_CATEGORICO = 5
categoricos = [] 
string = []
quantitativos = []
nao_numericos = []

for col in df:
    # print(col, df[col].unique())
    unicos = df[col].unique()
    ## dividindo colunas entre valores categoricos e quantitativos:
    if unicos.shape[0] < TH_CATEGORICO:
        categoricos.append(col)
    elif type(df[col][0]) is str:
        string.append(col)
    else: 
        quantitativos.append(col)
print(f'colunas categoricas: {len(categoricos)}, quantitativas: {len(quantitativos)}, str: {string}')

colunas categoricas: 0, quantitativas: 10, str: ['country', 'continent']


In [9]:
df[quantitativos].describe()

,total_confirmed,total_deaths,total_recovered,active_cases,serious_or_critical,total_cases_per_1m_population,total_deaths_per_1m_population,total_tests,total_tests_per_1m_population,population
count,1.440000e+02,144.000000,1.440000e+02,1.440000e+02,144.000000,144.000000,144.000000,1.440000e+02,1.440000e+02,1.440000e+02
mean,1.307081e+06,26832.638889,1.184411e+06,9.583686e+04,597.895833,48260.590278,806.881944,1.955644e+07,9.031496e+05,4.726722e+07
std,4.386056e+06,83171.849105,3.955494e+06,4.729728e+05,1632.836961,40317.198544,772.783946,6.523976e+07,1.677604e+06,1.704177e+08
min,5.890000e+02,3.000000,1.040000e+02,-1.489000e+03,1.000000,65.000000,3.000000,1.714700e+04,5.009000e+03,3.044800e+04
25%,3.404525e+04,507.000000,3.120325e+04,1.206000e+03,9.000000,10647.000000,123.000000,3.612762e+05,1.159155e+05,2.063909e+06
50%,1.970805e+05,2901.000000,1.695775e+05,9.814500e+03,36.500000,42805.000000,607.000000,2.082408e+06,4.277675e+05,9.227520e+06
75%,6.737082e+05,14695.000000,6.552955e+05,3.883925e+04,307.000000,80596.750000,1226.250000,1.142994e+07,1.015640e+06,3.396843e+07
max,3.573397e+07,629290.000000,3.081287e+07,5.438593e+06,9375.000000,162265.000000,3117.000000,5.295773e+08,1.311142e+07,1.439324e+09


In [10]:
# grellert: deprecated
def mean_of(data):
    _mean = 0
    _std = 0
    _median = 0
    _min_ = 0
    _max_ = 0
    for i in range(len(data)):
        _mean += data[i][0]
        _std += data[i][1]
        _median += data[i][2]
        _min_ += data[i][3]
        _max_ += data[i][4]
    mean = round(_mean/len(data), 4)
    std = round(_std/len(data), 4)
    median = round(_median/len(data), 4)
    min_ = round(_min_/len(data), 4)
    max_ = round(_max_/len(data), 4)
    return [mean, std, median, min_, max_]

In [11]:
def compare (df1, df2, class_data = None):
    mean= df2.mean() - df1.mean()
    std= df2.std() - df1.std()
    median = df2.median() - df1.median()
    minim = df2.min() - df1.min()
    maxim = df2.max() - df1.max()
    # @grellert: adicionando correlacao
    if class_data is not None:
        corr_diff = np.corrcoef(df2, class_data)[0][1] - np.corrcoef(df1, class_data)[0][1]
        return [mean, std, median, minim, maxim, corr_diff]
    return [mean, std, median, minim, maxim]

In [12]:
def imputation_test (k,df_,col,method):
    df = df_
    n_rows = df.shape[0]
    df_compare = []
    
    k_ = int(n_rows*k/100)
    to_nan = rd.sample(range(n_rows), k_)  # seleciona aleatorimente k% do numero de linhas total
    
    df[col].iloc[to_nan] = np.nan  

    if method == 'mean':
        df[col] = df[col].replace(np.nan, df[col].mean())

    elif method == 'median':
        imputer = SimpleImputer(strategy = 'median')
#         df[col] = df[col].replace(np.nan, df[col].median())
        imputer = imputer.fit(np.array(df[col]).reshape(-1,1))
        df[col] = imputer.transform(np.array(df[col]).reshape(-1,1))
        
    elif method == 'mode':
        df[col].fillna(df[col].mode()[0], inplace = True)
        
    # @grellert: tornando o método mais expansível
    elif method == 'knn' in method:
        k = method.split('_')[1]
        imputer = KNNImputer(n_neighbors = k)
        imputed_data = imputer.fit_transform(df)
        df_temp = pd.DataFrame(imputed_data)
        df_temp.columns = df.columns
        df[col] = df_temp[col]  # update only the desired column
        
    elif method == 'missforest':
        imputer = MissForest(verbose=0, n_estimators = 10, max_depth = 10, max_iter = 2 )
        imputed_data = imputer.fit_transform(df)
        df_temp = pd.DataFrame(imputed_data)
        df_temp.columns = df.columns
        df[col] = df_temp[col]
        
    elif method == 'iterative_imputer': ##still incorrect
        imp_mean = IterativeImputer()
        # recebe todo o DataFrame e retorna um np.array com todas as colunas com missing imputadas
        array_imp = imp_mean.fit_transform(df[quantitativos])
        df = pd.DataFrame(array_imp, columns = quantitativos)
        
    elif method == 'empty':
        return df
    else:
        print("method not available or invalid")
        return None
    return df
        

In [13]:
def test_column(df, col, method, class_col = None):
    compare_means = []
    compare_data = []
    k = [x for x in range(5,100,5)]
    for j in range(len(k)):
        for i in range(50):
            df_imp = df.copy()
            df_imp = imputation_test(k[j], df_imp, col, method)
            if df_imp is None:
                continue
            row = [method, k[j], col]
            row += compare(df[col], df_imp[col], class_data = None)
            compare_data.append(row)
            #print(compare_data)
        #compare_data = np.array(compare_data)
        #compare_means.append(compare_data.mean(axis = 0))
    if class_col:
        return pd.DataFrame(compare_data, columns = ['imputation method', 'pct imputed', 'attribute', 'Δmean', 'Δstd', 'Δmedian', 'Δmin', 'Δmax','Δcorrel'])
    return pd.DataFrame(compare_data, columns = ['imputation method', 'pct imputed', 'attribute', 'Δmean', 'Δstd', 'Δmedian', 'Δmin', 'Δmax' ])
    

In [14]:
def generate_results(df, method):
    df_nan = df.copy()  # df auxiliar utilizando apenas as colunas quantitativas
    data = [] # array que armazenará os dados do dataframe resultante
    data_ = []
    df_vis1 = [] # array que armazenará o df resultante de cada coluna de df_nan
    for col in quantitativos:
        print(col)
        data = test_column(df_nan, col, method)
        df_vis1.append(pd.DataFrame(data = data))
    print("done")
    df_vis = pd.concat(df_vis1)
    return df_vis

# Teste de imputação

In [15]:
df_vis1 = generate_results(df, 'empty')
# df_vis2 = generate_results(df, 'median')
# df_vis3 = generate_results(df, 'mode')
# df_vis = pd.concat([df_vis1, df_vis2, df_vis3])

total_confirmed
total_deaths
total_recovered
active_cases
serious_or_critical
total_cases_per_1m_population
total_deaths_per_1m_population
total_tests
total_tests_per_1m_population
population
done


In [17]:
def generate_vis(df_vis, coluna):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 30))
    
    sns.boxplot(ax=axes[0], x = 'pct imputed', y = 'Δmean', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette = 'Set2', showfliers = False); axes[0].set_title(coluna) 
    sns.boxplot(ax=axes[1], x = 'pct imputed', y = 'Δmedian', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette = 'Set2', showfliers = False); 
    sns.boxplot(ax=axes[2], x = 'pct imputed', y = 'Δstd', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette = 'Set2', showfliers = False); 
#     sns.boxplot(ax=axes[3], x = 'pct imputed', y = 'Δmin', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette = 'Pastel1', showfliers = False); 
#     sns.boxplot(ax=axes[4], x = 'pct imputed', y = 'Δmax', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette = 'Pastel1', showfliers = False); 
    sns.boxplot(ax=axes[3], x = 'pct imputed', y = 'Δcorrel', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette="Blues_d"); 
        
    axes[0].axhline(y=0, linestyle = '--')
    axes[1].axhline(y=0, linestyle = '--')
    axes[2].axhline(y=0, linestyle = '--')
#     axes[3].axhline(y=0, linestyle = '--')


In [48]:
coluna = []
for i in df[quantitativos]:
    coluna.append(i)
# b = sns.lmplot(x = df_vis1.columns[1], y = df_vis1.columns[3], hue = df_vis1.columns[2], data = df_vis1)
# a = [df_vis1['attribute']=='serious_or_critical']
# b = df_vis1[['attribute']=='total_deaths']
c = sns.lmplot(x = 'attribute', hue = 'pct imputed', data = df_vis1)

ValueError: 'total_deaths' is not in list

# Coluna Idade

In [ ]:
colunas = df_vis2['nome'].unique()
generate_vis(df_vis2, colunas[0])

# Coluna Vontade

In [ ]:
generate_vis(df_vis, colunas[1])

# Coluna Raiva

In [ ]:
generate_vis(df_vis, colunas[2])

# Coluna Medo

In [ ]:
generate_vis(df_vis, colunas[3])

# Coluna Cautela

In [ ]:
generate_vis(df_vis, colunas[4])

# Coluna Inibição

In [ ]:
generate_vis(df_vis, colunas[5])

# Coluna Sensibilidade

In [ ]:
generate_vis(df_vis, colunas[6])

# Coluna Coping

In [ ]:
generate_vis(df_vis, colunas[7])

In [ ]:
df_vis4 = generate_results(df, 'knn_2')
df_vis5 = generate_results(df, 'knn_3')
df_vis6 = generate_results(df, 'knn_4')
df_vis7 = generate_results(df, 'knn_5')

In [ ]:
df_vis = pd.concat([df_vis4, df_vis5, df_vis6, df_vis7])

In [ ]:
colunas = df_vis['nome'].unique()
generate_vis(df_vis, colunas[1])

In [ ]:
generate_vis(df_vis, colunas[2])

# Dados Categóricos - Teste de Imputação

In [ ]:
df[categoricos].describe()

In [ ]:
def cat_imputation_test(k,df,col,method):
    n_rows = df.shape[0]
    df_compare = []
    
    k_ = int(n_rows*k/100)
    to_nan = rd.sample(range(n_rows), k_)  # seleciona aleatorimente k% do numero de linhas total
    
    df[col].iloc[to_nan] = np.nan  

    if method == 'mode':
        df[col].fillna(df[col].mode()[0], inplace = True)
        
    elif method == 'knn' :
        imputer = KNNImputer(n_neighbors = 2)
        imputed_data = imputer.fit_transform(df)
        df_temp = pd.DataFrame(imputed_data)
        df_temp.columns = df.columns
        df[col] = df_temp[col]  # update only the desired column
        
    else:
        print("method not available or invalid")

In [ ]:
def cat_mean_of(data):
    _mean = 0
    _std = 0
    _median = 0
    _min_ = 0
    _max_ = 0
    for i in range(len(data)):
        _mean += data[i][0]
        _std += data[i][1]
        _median += data[i][2]
        _min_ += data[i][3]
        _max_ += data[i][4]
    mean = round(_mean/len(data), 4)
    std = round(_std/len(data), 4)
    median = round(_median/len(data), 4)
    min_ = round(_min_/len(data), 4)
    max_ = round(_max_/len(data), 4)
    return [mean, std, median, min_, max_]

In [ ]:
def cat_compare (df1, df2):
#     60 % M 40% F
#     moda
#     correlação ***somente por enquanto
#     mean = df2.mean() - df1.mean()
#     std = df2.std() - df1.std()
    median = df2.median() - df1.median()
#     minim = df2.min() - df1.min()
#     maxim = df2.max() - df1.max()
    return [mean, std, median, minim, maxim]

In [ ]:
def cat_test_column(df, col, method):
    data = []
    data_ = []
    k = [x for x in range(5,100,5)]
    df1 = df[col].copy()
    for j in range(len(k)):
        for i in range(50):
            cat_imputation_test(k[j], df, col, method)
            df2 = df[col].copy()
            data.append(cat_compare(df1, df2))
        data_.append(cat_mean_of(data))
        print(data_)
    return data_

In [ ]:
def cat_generate_results(df, method):
    df_nan = df[categoricos].copy()  # df auxiliar utilizando apenas as colunas quantitativas
    nao_numericos = []
    data = [] # array que armazenará os dados do dataframe resultante
    data_ = []
    k = [x for x in range(5,100,5)] # porcentagem de variáveis a serem retiradas aleatóriamente de cada coluna de df_nan
    df_vis1 = [] # array que armazenará o df resultante de cada coluna de df_nan
    
    for col in df_nan.columns:
#         if type(df_nan[col].loc[0]) is str:
#             nao_numericos.append(col)
#         else:
            print(col)
            data = cat_test_column(df_nan, col, method)
            d = {'imputation_method' : method,
              'nan values' : k,
              'nome' : col,
              'Δ mean' : [data[i][0] for i in range(19)],
              'Δ std' : [data[i][1] for i in range(19)],
              'Δ median' : [data[i][2] for i in range(19)],
              'Δ min' : [data[i][3] for i in range(19)],
              'Δ max' : [data[i][4] for i in range(19)]
                }
            df_vis1.append(pd.DataFrame(data = d))
    print("done")
    df_vis = pd.concat(df_vis1)
    return df_vis

In [ ]:
def cat_generate_vis(df_vis, coluna):
    fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(10,30))
    sns.barplot(ax=axes[0], x = 'pct imputed', y = 'Δmean', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette="Blues_d"); axes[0].set_title(coluna) 
    sns.barplot(ax=axes[1], x = 'pct imputed', y = 'Δmedian', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette="Blues_d"); 
    sns.barplot(ax=axes[2], x = 'pct imputed', y = 'Δstd', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette="Blues_d"); 
    sns.barplot(ax=axes[3], x = 'pct imputed', y = 'Δmin', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette="Blues_d"); 
    sns.barplot(ax=axes[4], x = 'pct imputed', y = 'Δmax', hue = 'imputation method', data = df_vis[df_vis['attribute'] == coluna], palette="Blues_d"); 

In [ ]:
df_vis1 = cat_generate_results(df, 'mode')

In [ ]:
colunas = df_vis1['nome'].unique()
cat_generate_vis(df_vis1, colunas[1])

In [ ]:
df_vis = generate_results(df, "missforest")
generate_vis(df, "missforest")

fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12,10))
sns.barplot(ax=axes[0], x='pct imputed', y='Δ std', hue='nome', data=df_vis); 
sns.barplot(ax=axes[1], x='pct imputed', y='Δ mean', hue='nome', data=df_vis); 
sns.barplot(ax=axes[2], x='pct imputed', y='Δ median', hue='nome', data=df_vis); 
sns.barplot(ax=axes[3], x='pct imputed', y='Δ max', hue='nome', data=df_vis); 

In [ ]:
def stats_ (col):
    df_pos = df.loc[df['THB'] == 'Sim']
    df_neg = df.loc[df['THB'] != 'Sim']
    var = col
    tstat, pval = stats.ttest_ind(df_pos[var], df_neg[var], nan_policy='omit')
    return tstat, pval